In [32]:
from rucio.client import Client
import itertools

import random
import string

In [21]:
client = Client()

RuntimeError: Could not load Rucio configuration file. Rucio looked in the following paths for a configuration file, in order:
	/opt/rucio/etc/rucio.cfg

# Initialize RSE

In [9]:
sites = {
    "T2_US_SDSC": {
        "hostname": "xrootd-sense-ucsd-redirector-111.sdsc.optiputer.net",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//store/temp"
        # "prefix": "//"
    },
    "T2_US_Caltech_Test": {
        "hostname": "sense-redir-01.ultralight.org",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//store/temp"
        # "prefix": "//store/data" 
    },
    "T1_US_FNAL": {
        "hostname": "cmssense4-origin-2841-1.fnal.gov",
        "scheme": "davs",
        "port": 2880,
        "prefix": "//dcache/store/temp"
    },
    "T2_US_Caltech": {
        "hostname": "redir-01.t2-sense.ultralight.org",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//store/temp"
    },
    "T2_US_Nebraska_Test": {
        "hostname": "red-sense-dtn1.unl.edu",
        "scheme": "davs",
        "port": 1094,
        "prefix": "//"
    },
}

In [10]:
for site in sites.values():
    site.update({"impl": "rucio.rse.protocols.gfal.Default",
                "domains": {
                    "lan": {
                        "read": 1,
                        "write": 1,
                        "delete": 1
                    },
                    "wan": {
                        "read": 1,
                        "write": 1,
                        "delete": 1,
                        "third_party_copy_read": 1,
                        "third_party_copy_write": 1
                    }
                }})

for site, site_info in sites.items():
    client.add_rse(site)
    client.add_protocol(site, site_info)
    client.set_local_account_limit("root", site, "-1")
    client.add_rse_attribute(site, "fts", "https://fts3-pilot.cern.ch:8446")
    client.add_rse_attribute(site, "lfn2pfn_algorithm", "cmstfc")
    client.add_rse_attribute(site, "verify_checksum", "false")

iterator = itertools.combinations(sites.keys(), 2)
for src, dst in iterator:
    client.add_distance(src, dst, {"ranking": 1, "distance": 1})
    client.add_distance(dst, src, {"ranking": 1, "distance": 1})

client.add_scope("root", "cms")

NameError: name 'client' is not defined

# Add Rule

In [29]:
PRIORITY = 5
DATASET = "65000,66000"
NUM_FILES_PER_DATASET = 1000
SOURCE = "T2_US_SDSC"
DESTINATION = "T2_US_Caltech"

In [31]:
container_suffix = ''.join(random.choices(string.ascii_uppercase, k=4))

datasets = DATASET.split(",")

for dataset in range(int(datasets[0]), int(datasets[1])+1000, 1000):
    files = [f"/store/data/Run2018A/EGamma/MINIAOD/UL2018_MiniAODv2-v1/{dataset}/testSourceFile{j}.root" for j in range(NUM_FILES_PER_DATASET)]

    dids = [{"scope": "cms", "name": f} for f in files]
    dids_w_bytes = [{"scope": "cms", "name": f, "bytes": 4294967296} for f in files]

    client.add_replicas(SOURCE, dids_w_bytes)

    client.add_dataset("cms", f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}")

    client.add_files_to_datasets([{"scope": "cms", "name": f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}", "dids": dids}])

client.add_container("cms", f"/SenseTest/Run3/{container_suffix}")

datasets_for_containers = [{"scope": "cms", "name": f"/SenseTest/Run2022-03Jan2023/MYDATA#{dataset}"} for dataset in range(int(datasets[0]), int(datasets[1])+1000, 1000)]
client.add_datasets_to_containers([{"scope": "cms", "name": f"/SenseTest/Run3/{container_suffix}", "dids": datasets_for_containers}])

rule = client.add_replication_rule(dids=[{"scope": "cms", "name": f"/SenseTest/Run3/{container_suffix}"}],
    copies=1,
    meta="sense",
    rse_expression=DESTINATION,
    priority=PRIORITY,
    source_replica_expression=SOURCE)

NameError: name 'client' is not defined